# Task: Get the last two years olhc price data for the top 5 ticker for every sector

In [3]:
!pip install scrapethat

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.8 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=3f200bd2915f3579785495e076fadc45680d6fda229ffa7a3a542ada4098263c
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


# https://cryptocointracker.com/yahoo-finance/yahoo-finance-api

### Getting Historical Prices from Yahoo Finance API #v8-finance-chart


In [6]:
from scrapethat import *
import requests
import json

In [ ]:
data_query = '{"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"exchange","operation":"in_range","right":["AMEX","NASDAQ","NYSE"]},{"left":"is_primary","operation":"equal","right":true},{"left":"active_symbol","operation":"equal","right":true}],"options":{"lang":"en"},"markets":["america"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","change","change_abs","Recommend.All","volume","Value.Traded","market_cap_basic","price_earnings_ttm","earnings_per_share_basic_ttm","number_of_employees","sector","industry","RSI","SMA50","SMA100","SMA200","BB.lower","BB.upper","Perf.W","Perf.Y","Perf.YTD","Perf.3M","Perf.6M","Perf.1M","High.1M","High.3M","High.6M","price_52_week_high","High.All","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","RSI[1]","currency","fundamental_currency_code"],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc"},"range":[0,9000]}'
response = requests.post('https://scanner.tradingview.com/america/scan', data=data_query)
data = response.json()
list_elements = list(map(lambda x:x['d'], data['data'] ))
stock = pd.DataFrame(list_elements)
stock.columns = json.loads(data_query)['columns']


In [ ]:
tickers = stock.groupby('sector').apply(lambda x: x.nlargest(5, 'market_cap_basic')).reset_index(drop=True)
tickers = tickers['name']

In [ ]:
ticker = 'TSLA'
range='18y'
interval='1d'

scraper = cloudscraper.create_scraper()
response = scraper.get(f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?interval={interval}&range={range}")
t= response.json()

df = pd.DataFrame(t['chart']['result'][0]['indicators']['quote'][0])
df['date'] = pd.to_datetime(t['chart']['result'][0]['timestamp'], unit='s').date
df = df[['date', 'open', 'low', 'high', 'close', 'volume']]

df

date        open         low        high       close     volume
0     2010-06-29    1.266667    1.169333    1.666667    1.592667  281494500
1     2010-06-30    1.719333    1.553333    2.028000    1.588667  257806500
2     2010-07-01    1.666667    1.351333    1.728000    1.464000  123282000
3     2010-07-02    1.533333    1.247333    1.540000    1.280000   77097000
4     2010-07-06    1.333333    1.055333    1.333333    1.074000  103003500
...          ...         ...         ...         ...         ...        ...
3367  2023-11-13  215.600006  211.610001  225.399994  223.710007  140447600
3368  2023-11-14  235.029999  230.720001  238.139999  237.410004  149771600
3369  2023-11-15  239.289993  236.449997  246.699997  242.839996  150354000
3370  2023-11-16  239.490005  230.960007  240.880005  233.589996  136816800
3371  2023-11-17  232.000000  226.539993  237.389999  234.300003  142532800

[3372 rows x 6 columns]

In [ ]:
def get_olhc(ticker, range='2y', interval='1d' ):
    try:

        scraper = cloudscraper.create_scraper()
        response = scraper.get(f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?interval={interval}&range={range}")
        t= response.json()
        df = pd.DataFrame(t['chart']['result'][0]['indicators']['quote'][0])
        df['date'] = pd.to_datetime(t['chart']['result'][0]['timestamp'], unit='s').date
        df = df[['date', 'open', 'low', 'high', 'close', 'volume']]
        df['name']= ticker
        return(df)

    except:
        print('error')
        pass





In [ ]:
get_olhc('TSLA')

date        open         low        high       close     volume  \
0    2021-11-18  368.850006  358.339996  370.666656  365.459991   62696700   
1    2021-11-19  366.290009  364.233337  379.573334  379.019989   64926900   
2    2021-11-22  387.443329  377.476654  400.649994  385.623322   99217500   
3    2021-11-23  389.170013  354.233337  393.500000  369.676666  108515100   
4    2021-11-24  360.130005  354.000000  377.589996  372.000000   67680600   
..          ...         ...         ...         ...         ...        ...   
498  2023-11-13  215.600006  211.610001  225.399994  223.710007  140447600   
499  2023-11-14  235.029999  230.720001  238.139999  237.410004  149771600   
500  2023-11-15  239.289993  236.449997  246.699997  242.839996  150354000   
501  2023-11-16  239.490005  230.960007  240.880005  233.589996  136816800   
502  2023-11-17  232.000000  226.539993  237.389999  234.300003  142532800   

     name  
0    TSLA  
1    TSLA  
2    TSLA  
3    TSLA  
4    TSLA  
..    ...  
498  TSLA  
499  TSLA  
500  TSLA  
501  TSLA  
502  TSLA  

[503 rows x 7 columns]

In [ ]:
from tqdm import tqdm

list_of_dfs = list(map(get_olhc, tqdm(tickers)))




 41%|████      | 41/100 [00:09<00:13,  4.44it/s]

error


100%|██████████| 100/100 [00:25<00:00,  3.98it/s]


In [ ]:
import pandas as pd

all_data = pd.concat(list_of_dfs, axis="rows").reset_index(drop=True)


In [ ]:

all_data

date        open         low        high       close    volume  \
0      2021-11-18  206.500000  201.270004  207.869995  203.330002  18292200   
1      2021-11-19  202.039993  199.850006  203.500000  200.860001  18032600   
2      2021-11-22  200.000000  194.600006  201.699997  195.580002  21185800   
3      2021-11-23  196.970001  194.380005  198.919998  198.490005  14742700   
4      2021-11-24  198.169998  197.639999  203.949997  203.250000  12026800   
...           ...         ...         ...         ...         ...       ...   
49792  2023-11-13   77.290001   74.970001   77.589996   75.019997   4549900   
49793  2023-11-14   76.750000   76.620003   77.989998   77.410004   3836900   
49794  2023-11-15   77.279999   76.440002   78.440002   76.529999   3404700   
49795  2023-11-16   77.480003   76.900002   78.080002   77.379997   4380100   
49796  2023-11-17   77.720001   76.870003   77.910004   77.389999   3507100   

      name  
0        V  
1        V  
2        V  
3        V  
4        V  
...    ...  
49792  AEP  
49793  AEP  
49794  AEP  
49795  AEP  
49796  AEP  

[49797 rows x 7 columns]